<a href="https://colab.research.google.com/github/tazar09/cars_dimensions/blob/main/comparison_31-ian2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings('ignore')

# Scrapping the website

In [20]:
url = 'https://www.automobiledimension.com/compact-suv.php'
h3 = requests.get(url, verify = False)
html_content = h3.text

# Creating BeautifulSoup objects

In [9]:
soup = BeautifulSoup(html_content, 'html.parser')

In [10]:
main_table = soup.find('main')

# Extract model description

In [11]:
temp_list = []
for i in main_table('a'):
  href = i['href']
  if href and href.startswith('/model'):
    temp_list.append(href)

In [12]:
weblink = []
for i in temp_list:
  weblink.append('https://www.automobiledimension.com' + i)

In [ ]:
models = []
for i in weblink:
  content = requests.get(i, verify = False)
  content_text = BeautifulSoup(content.text, 'html.parser')
  interior_text = content_text.find('div', class_ = 'interior-text')
  if interior_text:
    div_content = interior_text.text
    models.append(div_content)
  else:
    pass

In [14]:
models[0]

'The Ford Puma has a length of 4207 mm, a height of 1537 mm, a width of 1805 mm without the exterior mirrors and a measurement of 1930 millimeters with the mirrors unfolded. Motorization: petrol and  petrol mild hybrid. The mild hybrid model is called Puma Ecoboost Hybrid and has a boot space of 401* liters. Because of its size, features and ground clearance of 16 cm, we classify the Ford Puma in the category of compact SUV.Do you like the trend of larger cars? 👍 👎'

# Extract dimensions

## Function for extracting

In [49]:
def extract_car_info(text):
  pattern_sizes = r'\b[0-9]{1,4}\b'
  matches = re.findall(pattern_sizes, models[0])
  matches
  pattern_category = r"category\s+of\s+(.*?)\."
  match = re.search(pattern_category, models[0])
  if match:
    car_category = match.group(1)
  else:
    pass
  keys = ["length", "height", "width", "measurement", "boot space", "ground clearance"]
  values = matches
  my_dict = dict(zip(keys, values))
  my_dict['car category'] = car_category

  return list(my_dict.items())

In [50]:
extract_car_info(models[0])

[('length', '4207'),
 ('height', '1537'),
 ('width', '1805'),
 ('measurement', '1930'),
 ('boot space', '401'),
 ('ground clearance', '16'),
 ('car category', 'compact SUV')]

In [41]:
df = pd.DataFrame(models, columns = ['description'])
df.head()

,description
0,"The Ford Puma has a length of 4207 mm, a heigh..."
1,"The Audi Q2 has a length of 4208 mm, a height ..."
2,"The Nissan Juke has a length of 4210 mm, a hei..."
3,"The Opel Crossland has a length of 4212 mm, a ..."
4,"The SsangYong Tivoli has a length of 4225 mm, ..."


In [42]:
columns = ["length", "height", "width", "measurement", "boot space", "ground clearance"]
for i in columns:
  df[i]=df['description'].apply(lambda x: extract_car_info)

TypeError: 'function' object is not subscriptable

In [44]:
models[0].apply(lambda x: extract_car_data)

AttributeError: 'str' object has no attribute 'apply'